Notebook created: 2018-02-06 23:30:24  
Generated from: _build_jl/jl/mccall_model.rst  

In [1]:
using Distributions

# A default utility function

function u(c::Real, σ::Real)
    if c > 0
        return (c^(1 - σ) - 1) / (1 - σ)
    else
        return -10e6
    end
end

# default wage vector with probabilities

const n = 60                                   # n possible outcomes for wage
const default_w_vec = linspace(10, 20, n)   # wages between 10 and 20
const a, b = 600, 400                          # shape parameters
const dist = BetaBinomial(n-1, a, b)
const default_p_vec = pdf.(dist, support(dist))

mutable struct McCallModel{TF <: AbstractFloat,
                           TAV <: AbstractVector{TF},
                           TAV2 <: AbstractVector{TF}}
    α::TF        # Job separation rate
    β::TF         # Discount rate
    γ::TF        # Job offer rate
    c::TF            # Unemployment compensation
    σ::TF        # Utility parameter
    w_vec::TAV # Possible wage values
    p_vec::TAV2 # Probabilities over w_vec

    McCallModel(α::TF=0.2,
                β::TF=0.98,
                γ::TF=0.7,
                c::TF=6.0,
                σ::TF=2.0,
                w_vec::TAV=default_w_vec,
                p_vec::TAV2=default_p_vec) where {TF, TAV, TAV2} =
        new{TF, TAV, TAV2}(α, β, γ, c, σ, w_vec, p_vec)
end

"""
A function to update the Bellman equations.  Note that V_new is modified in
place (i.e, modified by this function).  The new value of U is returned.

"""
function update_bellman!(mcm::McCallModel, V::AbstractVector,
                         V_new::AbstractVector, U::Real)
    # Simplify notation
    α, β, σ, c, γ = mcm.α, mcm.β, mcm.σ, mcm.c, mcm.γ

    for (w_idx, w) in enumerate(mcm.w_vec)
        # w_idx indexes the vector of possible wages
        V_new[w_idx] = u(w, σ) + β * ((1 - α) * V[w_idx] + α * U)
    end

    U_new = u(c, σ) + β * (1 - γ) * U +
                    β * γ * dot(max.(U, V), mcm.p_vec)

    return U_new
end


function solve_mccall_model(mcm::McCallModel;
                            tol::AbstractFloat=1e-5, max_iter::Integer=2000)

    V = ones(length(mcm.w_vec))  # Initial guess of V
    V_new = similar(V)           # To store updates to V
    U = 1.0                        # Initial guess of U
    i = 0
    error = tol + 1

    while error > tol && i < max_iter
        U_new = update_bellman!(mcm, V, V_new, U)
        error_1 = maximum(abs, V_new - V)
        error_2 = abs(U_new - U)
        error = max(error_1, error_2)
        V[:] = V_new
        U = U_new
        i += 1
    end

    return V, U
end

solve_mccall_model (generic function with 1 method)

In [2]:
using Plots
gr()


mcm = McCallModel()
V, U = solve_mccall_model(mcm)
U_vec = U .* ones(length(mcm.w_vec))

plot(mcm.w_vec, 
     [V U_vec],
     lw=2, 
     α=0.7, 
     label=["V" "U"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 17.5 
 
 
 20.0 
 
 
 45.60 
 
 
 45.65 
 
 
 45.70 
 
 
 45.75 
 
<polyline clip-path="url(#clip9302)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 45.4014,375.813 54.4677,363.68 63.534,351.944 72.6004,340.587 81.6667,329.591 90.733,318.938 99.7994,308.613 108.866,298.601 117.932,288.887 126.998,279.46 
 136.065,270.306 145.131,261.413 154.197,252.77 163.264,244.368 172.33,236.196 181.396,228.245 190.463,220.506 199.529,212.97 208.595,205.63 217.662,198.479 
 226.728,191.508 235.794,184.712 244.861,178.084 253.927,171.617 262.993,165.306 272.06,159.145 281.126,153.13 290.192,147.254 299.259,141.513 308.325,135.903 
 317.391,130.419 326.458,125.056 335.524,119.812 344.59,114.681 353.657,109.661 362.723,104.748 371.789,99.9379 380.856,95.2282 389.922,90.6157 398.988,86.0972 
 408.055,81.6701 417.121,77.3315 426.187,73.0788 435.254,68.9094 444.32,64.8211 453.386,60.8114 462.453,56.878 471.519,53.0188 480.585,49.2318 489.652,45.515 
 498.718,41.8663 507.784,38.2839 516.851,34.7661 525.917,31.3112 534.983,27.9173 544.05,24.583 553.116,21.3067 562.182,18.0869 571.249,14.9222 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip9302)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 45.4014,284.995 54.4677,284.995 63.534,284.995 72.6004,284.995 81.6667,284.995 90.733,284.995 99.7994,284.995 108.866,284.995 117.932,284.995 126.998,284.995 
 136.065,284.995 145.131,284.995 154.197,284.995 163.264,284.995 172.33,284.995 181.396,284.995 190.463,284.995 199.529,284.995 208.595,284.995 217.662,284.995 
 226.728,284.995 235.794,284.995 244.861,284.995 253.927,284.995 262.993,284.995 272.06,284.995 281.126,284.995 290.192,284.995 299.259,284.995 308.325,284.995 
 317.391,284.995 326.458,284.995 335.524,284.995 344.59,284.995 353.657,284.995 362.723,284.995 371.789,284.995 380.856,284.995 389.922,284.995 398.988,284.995 
 408.055,284.995 417.121,284.995 426.187,284.995 435.254,284.995 444.32,284.995 453.386,284.995 462.453,284.995 471.519,284.995 480.585,284.995 489.652,284.995 
 498.718,284.995 507.784,284.995 516.851,284.995 525.917,284.995 534.983,284.995 544.05,284.995 553.116,284.995 562.182,284.995 571.249,284.995 580.315,284.995 
 
 "/>
 
 
 
 
 V 
 
 
 
 U

In [3]:
"""
Computes the reservation wage of an instance of the McCall model
by finding the smallest w such that V(w) > U.

If V(w) > U for all w, then the reservation wage w_bar is set to
the lowest wage in mcm.w_vec.

If v(w) < U for all w, then w_bar is set to np.inf.

Parameters
----------
mcm : an instance of McCallModel
return_values : bool (optional, default=false)
    Return the value functions as well

Returns
-------
w_bar : scalar
    The reservation wage

"""
function compute_reservation_wage(mcm::McCallModel; return_values::Bool=false)

    V, U = solve_mccall_model(mcm)
    w_idx = searchsortedfirst(V - U, 0)

    if w_idx == length(V)
        w_bar = Inf
    else
        w_bar = mcm.w_vec[w_idx]
    end

    if return_values == false
        return w_bar
    else
        return w_bar, V, U
    end

end

compute_reservation_wage

In [4]:
grid_size = 25
γ_vals = linspace(0.05, 0.95, grid_size)

0.05:0.0375:0.95

In [14]:
using Plots, LaTeXStrings
gr()


grid_size = 25  
c_vals = linspace(2, 12, grid_size)  
w_bar_vals = similar(c_vals)

mcm = McCallModel()

for (i, c) in enumerate(c_vals)
    mcm.c = c
    w_bar = compute_reservation_wage(mcm)
    w_bar_vals[i] = w_bar
end

plot(c_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="unemployment compensation",
     ylabel="reservation wage",
     label="w bar as a function of c")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 12 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 14 
 
 
 unemployment compensation 
 
 
 reservation wage 
 
<polyline clip-path="url(#clip5402)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 44.6753,360.065 66.9936,360.065 89.312,360.065 111.63,360.065 133.949,360.065 156.267,360.065 178.585,360.065 200.904,324.039 223.222,300.022 245.54,263.995 
 267.859,239.978 290.177,215.96 312.495,191.943 334.813,179.934 357.132,155.916 379.45,143.908 401.768,119.89 424.087,107.881 446.405,83.8637 468.723,71.8549 
 491.042,59.8461 513.36,47.8373 535.678,35.8286 557.997,23.8198 580.315,11.811 
 "/>
 
 
 
 
 w bar as a function of c

In [13]:
grid_size = 25  
γ_vals = linspace(0.05, 0.95, grid_size)  
w_bar_vals = similar(γ_vals)

mcm = McCallModel()

for (i, γ) in enumerate(γ_vals)
    mcm.γ = γ
    w_bar = compute_reservation_wage(mcm)
    w_bar_vals[i] = w_bar
end

plot(γ_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="job offer rate",
     ylabel="reservation wage",
     label="w bar as a function of \\gamma")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 10.0 
 
 
 10.5 
 
 
 11.0 
 
 
 11.5 
 
 
 12.0 
 
 
 job offer rate 
 
 
 reservation wage 
 
<polyline clip-path="url(#clip5202)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 54.5598,360.065 76.4662,360.065 98.3727,360.065 120.279,360.065 142.186,360.065 164.092,360.065 185.999,360.065 207.905,360.065 229.811,360.065 251.718,360.065 
 273.624,306.488 295.531,279.699 317.437,252.91 339.344,226.121 361.25,199.333 383.157,172.544 405.063,145.755 426.97,118.966 448.876,118.966 470.783,92.1774 
 492.689,65.3886 514.596,65.3886 536.502,38.5998 558.408,11.811 580.315,11.811 
 "/>
 
 
 
 
 w bar as a function of 
 
 
 γ